In [ ]:
import numpy as np

**Question:** Look forward to determine whether we need `LUdecomp5solver()`. If that routine is not used later, we can remove half of this notebook.

# Day 6: Symmetric and Banded Coefficient Matrices

Applications often involve coefficient matrices that are *sparse*ly populated. That means most of their entries are $0$'s. If all of the non-zero entries are clustered near the main diagonal, then the matrix is said to be *banded*. For example, see the matrix below, which is a tri-diagonal, banded matrix.

$$\left[\begin{array}{ccccccc}X & X & 0 & 0 & \cdots & 0 & 0 & 0\\
X & X & X & 0 & \cdots & 0 & 0 & 0\\
0 & X & X & X & \cdots & 0 & 0 & 0\\[10pt]
\vdots & \vdots & \vdots & \ddots & \ddots & \ddots & \vdots & \vdots\\[10pt]
0 & 0 & 0 & 0 &\cdots & X & X & X\\
0 & 0 & 0 & 0 & \cdots & 0 & X & X
\end{array}\right]$$

Those $X$ entries could be zero or non-zero, but all entries off of those three diagonals are $0$'s in a tri-diagonal matrix. We say that such a matrix has a *bandwidth* of $3$ since there are at most three non-zero elements in each row.

A nice property of these banded matrices is that, if they are *LU-decomposed*, then both $L$ and $U$ retain the banded structure. This structure can be exploited to save on both *memory requirements* and *run time*.

## An Example

To motivate our discovery and implementation of this method, let's try solving a system with a tridiagonal coefficient matrix.

**Example:** Solve the system
$$A = \left[\begin{array}{ccccc} 1 & 2 & 0 & 0 & 0\\
2 & -1 & 8 & 0 & 0\\
0 & 3 & -1 & -1 & 0\\
0 & 0 & 3 & 2 & -1\\
0 & 0 & 0 & 5 & -4\end{array}\right]\left[\begin{array}{c} x_1\\ x_2\\ x_3\\ x_4\\ x_5\end{array}\right] = \left[\begin{array}{c}13\\ 30\\ 7\\ 12\\ 6\end{array}\right]$$

> *Solution.*

### Doolittle's Method for Tridiagonal Coefficient Matrices

Consider the system $A\vec{x} = \vec{b}$, where $A$ is a tridiagonal matrix of the following form:

$$A = \left[\begin{array}{cccccc} d_1 & e_1 & 0 & 0 & \cdots & 0\\
c_1 & d_2 & e_2 & 0 & \cdots & 0\\
0 & c_2 & d_3 & e_3 & \cdots & 0\\
0 & 0 & c_3 & d_4 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & 0 & c_{n-1} & d_n
\end{array}\right]$$

In this case, the majority of the entries will be $0$ (as long as the matrix is large enough). It becomes more efficient to store the three diagonal vectors only, since we know that all other entries are $0$'s. That is, instead of storing $A$ explicitly, we'll store:

$$\vec{c} = \left[\begin{array}{c} c_1\\ c_2\\ \vdots\\ c_{n-1}\end{array}\right]~~~\vec{d} = \left[\begin{array}{c} d_1\\ d_2\\ \vdots\\ d_n\end{array}\right]~~~\vec{e} = \left[\begin{array}{c} e_1\\ e_2\\ \vdots\\ e_{n-1}\end{array}\right]$$

This is a significant savings, since a $100\times 100$ tridiagonal matrix has $10,000$ entries, but we can get away with storing only $99 + 100 + 99 = 298$ entries using this vector storage trick. This is a *compression* of around $33:1$.

We can still apply *LU decomposition* to the coefficient "matrix" (really, the coefficient vectors) as long as we carefully track which vectors we are working with. As a reminder, the *LU-decomposition* begins with the usual *Gaussian Elimination* procedure. That is, to reduce $R_k$ (row $k$) we eliminate $c_{k-1}$ using:

$$R_k \leftarrow R_k - \left(c_{k-1}/d_{k-1}\right)R_{k-1}$$

Notice that in doing this, only $\vec{c}$ and $\vec{d}$ are changed. The changes are:

$$\begin{array}{lcl} d_k &\leftarrow &d_k - \left(c_{k-1}/d_{k-1}\right)e_{k-1}\\
c_{k-1} &\leftarrow & 0
\end{array}$$

At the end of the process, the vector $\vec{c}$ would be zeroed out. It is a waste of space to store that zero-vector, but $\vec{c}$ is a perfect place to store our $\lambda$ scalar multipliers for our pivot rows. For this reason, we'll update:

$$\begin{array}{lcl} d_k &\leftarrow &d_k - \left(c_{k-1}/d_{k-1}\right)e_{k-1}\\
c_{k-1} &\leftarrow & c_{k-1}/d_{k-1}
\end{array}$$

Thus, we have the following decomposition algorithm:

```
#tridiagonal LU decomp
for k in range(1, n):
  lam = c[k-1]/d[k-1]
  d[k] = d[k] - lam*e[k-1]
  c[k-1] = lam
```

Now we'll look to the *solution phase*. As a reminder, we're solving $A\vec{x} = \vec{b}$ by solving $LU\vec{x} = \vec{b}$. The strategy is to use forward-substitution to solve $L\vec{y} = \vec{b}$ and then backward-substitution to solve $U\vec{x} = \vec{y}$ as before, but we need to deal with the fact that our factorized matrix elements are split across the three vectors $\vec{c}$, $\vec{d}$, and $\vec{e}$.

We can solve $L\vec{y} = \vec{b}$ as follows (remember that $\vec{c}$ contains our $\lambda$ multipliers):

```
#forward substitution
y[0] = b[0]
for k in range(1, n):
  y[k] = b[k] - c[k-1]*y[k-1]
```

Now we solve $U\vec{x} = \vec{y}$ using backward-substitution.

```
#backward substitution
x[n-1] = y[n-1]/d[n-1]
for k in range(n-2, -1, -1):
  x[k] = (y[k] - e[k]*x[k+1])/d[k]
```

We'll put this all together into a `DoolittleLUdecomp3solver()` routine below. The routine should make use of two *helper functions*, `DoolittleLUdecomp3()` and `Doolittle3solver()`, which are described below.

+ `DoolittleLUdecomp3()` should take three parameters -- arrays `c`, `d`, and `e` representing the diagonals of the tridiagonal coefficient matrix as described above, and should return the transformed versions of `c`, `d`, and `e` resulting from the row reduction operations.
+ `Doolittle3solver()` should take four parameters -- a scalar `lam` and the vectors `c`, `d`, and `e` resulting from the `DoolittleLUdecomp3()` function. This function should return the solution vector for $A\vec{x} = \vec{b}$.
+ `DoolittleLUdecomp3solver()` should take four parameters -- arrays `c`, `d`, and `e` representing the diagonals of the tridiagonal coefficient matrix and the constant array `b`. The function should make use of `DoolittleLUdecomp3()` and `Doolittle3solver()` to return the solution of the matrix equation $A\vec{x} = \vec{b}$, where $A$ is a tridiagonal matrix with lower diagonal `c`, main diagonal `d`, and upper diagonal `e`.

In [1]:
#Define your function and helper functions here.


Verify that this solver works on our example from earlier!

In [ ]:
#Solve the example problem here


## Symmetric Coefficient Matrices

Because of how often applications result in symmetric, banded coefficient matrices (where $\vec{c} = \vec{e}$), mathematicians and engineers spend time developing specialized approaches that work more quickly and efficiently than general approaches. For example, it can be shown that, if a matrix $A$ is symmetric, then its *LU-decomposition* can be written as

$$A = LU = LDL^T$$

where $D$ is a *diagonal matrix*. This means that we can still use Doolittle's decomposition with $U = DL^T$. That is,

\begin{align*} U &= DL^T\\
&= \left[\begin{array}{ccccc} D_1 & 0 & 0 & \cdots & 0\\
0 & D_2 & 0 & \cdots & 0\\
0 & 0 & D_3 & \cdots & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]\left[\begin{array}{ccccc} 1 & L_{21} & L_{31} & \cdots & L_{n1}\\
0 & 1 & L_{32} & \cdots & L_{n2}\\
0 & 0 & 1 & \cdots & L_{n3}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & 1
\end{array}\right]\\
&= \left[\begin{array}{ccccc} D_1 & D_1L_{21} & D_1L_{31} & \cdots & D_1L_{n1}\\
0 & D_2 & D_2L_{32} & \cdots & D_2L_{n2}\\
0 & 0 & D_3 & \cdots & D_3L_{n3}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]
\end{align*}

Again, as a space-saving efficiency, we can choose to only store $U$ since both $L$ and $D$ are recoverable from this matrix! Again, the *Gaussian Elimination* procedure which results in an upper triangular matrix is sufficient to decompose a symmetric matrix.

There is an alternative, however, which is even more efficient due to speed-ups at the solution phase. We can store

$$U^* = \left[\begin{array}{ccccc} D_1 & L_{21} & L_{31} & \cdots & L_{n1}\\
0 & D_2 & L_{32} & \cdots & L_{n2}\\
0 & 0 & D_3 & \cdots & L_{n3}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]$$

and notice that $U_{ij} = D_iL_{ji} = U^*_{ii}*U^*_{ij}$. Since this is the case, we'll choose to store and utilize $U^*$ through our numerical routine.

***

## Optional: Symmetric, Pentadiagonal Coefficient Matrices

We often encounter bandwidth $5$ coefficient matrices when we attempt numerical approaches to solving fourth-order ODEs by finite difference methods. These matrices are of the form

$$A = \left[\begin{array}{cccccccc} d_1 & e_1 & f_1 & 0 & 0 & 0 & \cdots & 0\\
e_1 & d_2 & e_2 & f_2 & 0 & 0 & \cdots & 0\\
f_1 & e_2 & d_3 & e_3 & f_3 & 0 & \cdots & 0\\
0 & f_2 & e_3 & d_4 & e_4 & f_4 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots\\
0 & \cdots & 0 & f_{n-4} & e_{n-3} & d_{n-3} & d_{n-2} & f_{n-2}\\
0 & \cdots & 0 & 0 & f_{n-3} & e_{n-2} & d_{n-1} & e_{n-1}\\
0 & \cdots & 0 & 0 & 0 & f_{n-2} & e_{n-1} & d_n
\end{array}\right]$$

As in the case of our tridiagonal matrices, we'll store the non-zero elements in three vectors:

$$\vec{d} = \left[\begin{array}{c} d_1\\ d_2\\ \vdots\\ d_{n-2}\\ d_{n-1}\\ d_{n}\end{array}\right]~~~~ \vec{e} = \left[\begin{array}{c} e_1\\ e_2\\ \vdots\\ e_{n-2}\\ e_{n-1}\end{array}\right]~~~~ \vec{f} = \left[\begin{array}{c} f_1\\ f_2\\ \vdots\\ f_{n-2}\end{array}\right]$$

We'll again use Doolittle's decomposition to transform $A$ to upper triangular form by *Gaussian Elimination*. Consider the stage where the $k^{th}$ row has become the pivot row. Then, we have

$$\left[\begin{array}{cc|ccc|cccc} \ddots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \\ \hline
\cdots & 0 & d_k & e_k & f_k & 0 & 0 & 0 & \cdots \\
\cdots & 0 & e_k & d_{k+1} & e_{k+1} & f_{k+1} & 0 & 0 & \cdots \\
\cdots & 0 & f_k & e_{k+1} & d_{k+2} & e_{k+2} & f_{k+2} & 0 & \cdots\\ \hline
\cdots & 0 & 0 & f_{k+1} & e_{k+2} & d_{k+3} & e_{k+3} & f_{k+3} & \cdots\\
 & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots
\end{array}\right]$$

We eliminate $e_k$ in $R_{k+1}$ using

$$R_{k+1} \leftarrow R_{k+1} - \left(e_k/d_k\right)R_k$$

Similarly we eliminate $f_k$ in $R_{k+2}$ using

$$R_{k+2} \leftarrow R_{k+2} - \left(f_k/d_k\right)R_k$$

Other than the entries being zeroed out, the only terms being changed by this process are:

\begin{align*} d_{k+1} &\leftarrow d_{k+1} - \left(e_k/d_k\right)e_k\\
e_{k+1} &\leftarrow e_{k+1} - \left(e_k/d_k\right)f_k\\
d_{k+2} &\leftarrow d_{k+2} - \left(f_k/d_k\right)f_k
\end{align*}

Recalling that we are constructing $U^*$, which stores the $\lambda$ multipliers above the main diagonal, we can rewrite $e_k$ and $f_k$ with those corresponding multipliers. That is,

\begin{align*} e_k &\leftarrow e_k/d_k\\
f_k &\leftarrow f_k/d_k
\end{align*}

Once this is done, we'll have constructed a matrix of the following form (note $\vec{d}$, $\vec{e}$, and $\vec{f}$ are not the original vectors we began with).

$$U^* = \left[\begin{array}{cccccc} d_1 & e_1 & f_1 & 0 & \cdots & 0\\
0 & d_2 & e_2 & f_2 & \cdots & 0\\
0 & 0 & d_3 & e_3 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & 0 & d_{n-1} & e_{n-1}\\
0 & 0 & \cdots & 0 & 0 & d_n
\end{array}\right]$$

We now enter the solution phase, where we again solve $LU\vec{x} = \vec{b}$ by first solving $L\vec{y} = \vec{b}$ and then $U\vec{x} = \vec{y}$. Note that $L\vec{y} = \vec{b}$ has the form below.

$$\left[L\mid \vec{b}\right] = \left[\begin{array}{cccccc|c} 1 & 0 & 0 & 0 & \cdots & 0 & b_1\\
e_1 & 1 & 0 & 0 & \cdots & 0 & b_2\\
f_1 & e_2 & 1 & 0 & \cdots & 0 & b_3\\
0 & f_2 & e_3 & 1 & \cdots & 0 & b_4\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & \cdots & f_{n-2} & e_{n-1} & 1 & b_n\\
\end{array}\right]$$

Which can be solved using forward substitution, as below:

```
#forward substitution
#b[0] = b[0] #unnecessary
b[1] = b[1] - e[0]*b[0]
for k in range(2, n):
  b[k] = b[k] - (e[k-1]*b[k-1]) - (f[k-2]*b[k-2])
```

The augmented coefficient matrix corresponding to $U\vec{x} = \vec{y}$ has the form below.

$$\left[U\mid \vec{y}\right] = \left[\begin{array}{cccccc|c} d_1 & d_1e_1 & d_1f_1 & 0 & \cdots & 0 & y_1\\
0 & d_2 & d_2e_2 & d_2f_2 & \cdots & 0 & y_2\\
0 & 0 & d_3 & d_3e_3 & \cdots & 0 & y_3\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & \cdots & 0 & d_{n-1} & d_{n-1}e_{n-1} & y_{n-1}\\
0 & 0 & \cdots & 0 & 0 & d_n & y_n\\
\end{array}\right]$$

Again, this can be solved using back substitution as follows:

```
b[n-1] = b[n-1]/d[n-1]
b[n-2] = (b[n-2]/d[n-2]) - (e[n-2]*b[n-1])

for k in range(n-3, -1, -1):
  b[k] = b[k]/d[k] - (e[k]*b[k+1]) - (f[k]*b[k+2])
```

Finally, we can define the `DoolittleLUdecomp5solver()` below!


In [ ]:
def DoolittleLUdecomp5(d, e, f):
  n = len(d)
  for k in range(n - 2):
    lam = e[k]/d[k]
    d[k+1] = d[k+1] - lam*e[k]
    e[k+1] = e[k+1] - lam*f[k]
    e[k] = lam
    lam = f[k]/d[k]
    d[k+2] = d[k+2] - lam*f[k]
    f[k] = lam

  lam = e[n-2]/d[n-2]
  d[n-1] = d[n-1] - lam*e[n-2]
  e[n-2] = lam

  return d, e, f

def Doolittle5solver(d, lam_e, lam_f, b):
  n = len(d)
  b[1] = b[1] - (lam_e[0]*b[0])
  for k in range(2, n):
    b[k] = b[k] - (lam_e[k-1]*b[k-1]) - (lam_f[k-2]*b[k-2])

  b[n-1] = b[n-1]/d[n-1]
  b[n-2] = b[n-2]/d[n-2] - (lam_e[n-2]*b[n-1])
  for k in range(n-3, -1, -1):
    b[k] = b[k]/d[k] - (lam_e[k]*b[k+1]) - (lam_f[k]*b[k+2])

  return b

def DoolittleLUdecomp5solver(d, e, f, b):
  d, lam_e, lam_f = DoolittleLUdecomp5(d, e, f)
  x = Doolittle5solver(d, lam_e, lam_f, b)

  return x

We'll return to this `DoolittleLUdecomp5solver()` function later in our course. For now, we can use it to solve the following system (just to make sure our routine works!).

**Example:** Use the routine we wrote to solve the following symmetric, pentadiagonal linear system $$\left[\begin{array}{ccccccc} 2 & -3 & 1 & 0 & 0 & 0 & 0\\
-3 & 1 & 4 & -2 & 0 & 0 & 0\\
1 & 4 & -6 & -1 & 1 & 0 & 0\\
0 & -2 & -1 & 5 & 4 & 2 & 0\\
0 & 0 & 1 & 4 & 3 & 5 & -3\\
0 & 0 & 0 & 2 & 5 & 2 & 1\\
0 & 0 & 0 & 0 & -3 & 1 & 4
\end{array}\right]\left[\begin{array}{c} x_1\\ x_2\\ x_3\\ x_4\\ x_5\\ x_6\\ x_7\end{array}\right] = \left[\begin{array}{c} -5\\ 3\\ 2\\ -11\\ 4\\ 3\\ 1\end{array}\right]$$

In [ ]:
#Solve the example problem here


***

## Summary

In this notebook, we considered additional efficiencies that can be gained by exploiting properties of the coefficient matrix. In particular, we looked at two special cases of banded matrices and banded, symmetric matrices. We could construct specialty solvers for other classes of coefficient matrix by analyzing the process involved in using the *Gaussian Elimination* approach to solving the corresponding system. Taking an algorithmic approach is much slower for a single problem, but pays off because we end up with a constructed routine that can "quickly" solve any problem fitting the assumptions of the algorithm we've constructed.